<a href="https://colab.research.google.com/github/luandaoduy96/digitrecognization/blob/master/digitrecognization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse 
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

In [0]:
# Load the data
train = pd.read_csv("drive/My Drive/train.csv")
test = pd.read_csv("drive/My Drive/test.csv")

train.describe()

In [0]:
train.head()
train.columns

In [0]:
Y_train = train['label'] # nho dataframe['...']
Y_train.head()
X_train = train.drop('label',axis=1)
X_train.columns # X_train don't have label column

X_test = test

In [0]:
# check out missing data
null = train.isnull().sum()
null= null[null>0]
print(null) # no miss data worderful!!


In [0]:
# normalization data to compute faster
X_train = X_train/255
X_test = X_test/255


In [0]:
# reshape image to input layer
X_train=X_train.values.reshape(-1,28,28,1)
X_test=X_test.values.reshape(-1,28,28,1)

In [0]:
print("X_train ndim: ", X_train.ndim)
print("X_train shape:", X_train.shape)
print("X_train size: ", X_train.size)
print("the number of training examples = %i" % X_train.shape[0])
#print("the number of class= %i" % len(numpy.unique(Y_train)))



In [0]:
Y_train = to_categorical(Y_train, num_classes = 10)


In [0]:
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)


In [0]:
print(X_train.shape[0],Y_train.shape[0])

In [0]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Valid', activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Valid', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Valid', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Valid', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))


In [0]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy']) # cross entropy cost function is used and adam gradient descent is used
model.fit(X_train, Y_train, epochs= 30 , batch_size=200)
scores = model.evaluate(X_val, Y_val, verbose = 10 )
print ( scores )